# Deploy a scikit-learn machine learning model to IBM Cloud



This notebook explains how to deploy a scikit-learn model to IBM Cloud usign the **watson-machine-learning-client** library that allows you to interact with the IBM Machine Learning service.

### Goal
Deploy a **Linear Regression** model that predicts the median house price in a given area in Boston. 

### Data
We will use the **Boston House Prices dataset** provided with scikit-learn. The data consists of the following columns
    - CRIM     per capita crime rate by town
    - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
    - INDUS    proportion of non-retail business acres per town
    - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
    - NOX      nitric oxides concentration (parts per 10 million)
    - RM       average number of rooms per dwelling
    - AGE      proportion of owner-occupied units built prior to 1940
    - DIS      weighted distances to five Boston employment centres
    - RAD      index of accessibility to radial highways
    - TAX      full-value property-tax rate per $10,000.
    
    - PTRATIO  pupil-teacher ratio by town
    - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
    - LSTAT    % lower status of the population
    - MEDV     Median value of owner-occupied homes in $1000's
    
Our goal is to predict `MEDV` assuming we have the other features/inputs available.

### Typical Steps in Data Science Projects
0. Busines problem / understanding
1. Data acquisition
2. Explore the data / or use Data Refinery
3. Model preparation
4. Model evaluation and performance optimization
5. Store the model on IBM Cloud.
6. Deploy the model on IBM Cloud.
7. Use the model deployed on IBM Cloud.



## Pre-requisites
1. Create a Machine Learning service instance (use the free plan)
https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/wml-setup.html
2. This notebook needs 
   - Python 3.5
   - scikit-learn 0.19.1
   - watson-machine-learning-client

In [ ]:
!pip install watson-machine-learning-client --upgrade
# if this is already installed, you will see a whole bunch of `Requirement already up-to-date`

## 1. Load the data

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target)

# Create a new Linear Regression Model
LR_model = LinearRegression()

# Train the model
LR_model.fit(X_train, y_train)

# store actual and predited data to draw chart
predicted = LR_model.predict(X_test)
actual = y_test

plt.figure(figsize=(4, 3))
plt.scatter(actual, predicted)
plt.plot([0, 50], [0, 50], '--k')
plt.axis('tight')
plt.xlabel('True Median Value ($1000s)')
plt.ylabel('Predicted Median Value ($1000s)')
plt.tight_layout()

# The coefficients
print('Coefficients: \n', LR_model.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(actual, predicted))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(actual, predicted))

In [ ]:
# we will use WML to work with IBM Machine Learning Service
from watson_machine_learning_client import WatsonMachineLearningAPIClient

# Grab your credentials from the Watson Service section in Watson Studio or IBM Cloud Dashboard
wml_credentials = {
}

# Instantiate WatsonMachineLearningAPIClient
from watson_machine_learning_client import WatsonMachineLearningAPIClient
client = WatsonMachineLearningAPIClient( wml_credentials )

# store the model
published_model = client.repository.store_model(model=LR_model,
                                                meta_props={'name':'upkar-housing-linear-reg'},
                                                training_data=X_train, training_target=y_train)

In [ ]:
import json

# grab the model from IBM Cloud
published_model_uid = client.repository.get_model_uid(published_model)

# create a new deployment for the model
model_deployed = client.deployments.create(published_model_uid, "Deployment of scikit model")

#get the scoring endpoint
scoring_endpoint = client.deployments.get_scoring_url(model_deployed)
print(scoring_endpoint)

#use the scoring endpoint to predict house median price some test data
scoring_payload = {"values": [list(X_test[0]), list(X_test[1])]}
predictions = client.deployments.score(scoring_endpoint, scoring_payload)
print(json.dumps(predictions, indent=2))